In [1]:
import pandas as pd
# 读 property 数据， 改路径
property_data = pd.read_csv('property_data.csv')

In [2]:
import re
def extract_cost_number (row):
    cost_string = row['cost_text']
    
    # 替换 / 为 per
    cost_string = cost_string.replace('/', ' per ')
    
    cost_list = []
    
    # 找出可能的租金字符串
    for cost in re.findall('\d+\s*\w+\s*\w+',cost_string):
        cost_list.append(cost)
        
    cost_list = [cost.lower() for cost in cost_list]
    
    # 替换简写
    dictOfStrings = {'pw': 'per week', 'weekly': 'week',
                    'pcm':'per month','mth':'month','wek':'week',
                    'pa':'per year','wk':'week','annually':'year',
                    'pwk':'per week','poa':'per year','monthly':'month',
                    'pcw':'per week','weekly':'week','annum':'year','pm':'per month'}
    replace_abb = []
    for cost in cost_list:
        for word, replacement in dictOfStrings.items():
            cost = re.sub(word, replacement, cost)
        replace_abb.append(cost)     
    if len(replace_abb) > 0:
        cost_type = replace_abb[0]
        # 替换单字母简写
        dictOfAlpha = {'h':'hour','d':'day','w':'week','m':'month','y':'year'}
        processed_test = ''
        for w in cost_type.split():
            if (len(w) == 1) and (w in dictOfAlpha):
                processed_test += dictOfAlpha[w]
            else:
                processed_test += w
            processed_test += " "
        rent_type = ''.join([i for i in processed_test if not i.isdigit()])
        cost = ''.join([i for i in processed_test if i.isdigit()])
        if 'hour' in rent_type:
            rent_per_week = float(cost)*24*7
        elif 'day' in rent_type:
            rent_per_week = float(cost)*7    
        elif 'month' in rent_type:
            rent_per_week = (float(cost)/30)*7   
        elif 'year' in rent_type:
            rent_per_week = (float(cost)/365)*7  
        elif 'fortnightly' in rent_type:
            rent_per_week = (float(cost)/2)  
        else:
            rent_per_week = float(cost)
        return rent_per_week
    else:
        return "No Available"    

In [3]:
property_data['week_rent'] = property_data.apply (lambda row: extract_cost_number(row), axis=1)